In [1]:
import os
import sys
import pandas
import numpy

import findspark
findspark.init("/usr/local/spark/spark")

import pyspark
from pyspark.sql.window import Window
import pyspark.sql.functions as func

from pyspark.rdd import reduce
from pyspark.sql.types import DoubleType
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.clustering import GaussianMixture, KMeansModel, KMeans
from pyspark.ml.linalg import SparseVector, VectorUDT, Vector, Vectors

In [2]:
from pyspark.sql.functions import udf, col, struct
from pyspark.sql.types import ArrayType, DoubleType, StringType
from pyspark.mllib.linalg.distributed import RowMatrix, DenseMatrix

In [3]:
cluster_files =  "/Users/simondi/PROJECTS/target_infect_x_project/src/tix-analysis/data/mahalanobis/"

In [4]:
conf = pyspark.SparkConf().setMaster("local[*]").set("spark.driver.memory", "4G").set("spark.executor.memory", "4G")
sc = pyspark.SparkContext(conf=conf)
spark = pyspark.sql.SparkSession(sc)

In [5]:
df = spark.read.parquet(cluster_files)

In [6]:
df.take(1)

[Row(study='infectx_published', pathogen='listeria', library='a', design='u', replicate='1', plate='kb2-02-1x', well='a01', gene='chka', sirna='s3008', well_type='sirna', image_idx='4', object_idx='144', cells_areashape_area=-0.8044235706329346, cells_areashape_eccentricity=0.012107263319194317, cells_areashape_extent=1.115902304649353, cells_areashape_formfactor=1.474928617477417, cells_areashape_majoraxislength=-0.9368530511856079, cells_areashape_minoraxislength=-0.7485320568084717, cells_areashape_perimeter=-1.0209269523620605, cells_location_center_x=0.44758084416389465, cells_location_center_y=1.2808903455734253, cells_neighbors_anglebetweenneighbors_2=1.3915760517120361, cells_neighbors_firstclosestobjectnumber_2=0.1488679200410843, cells_neighbors_firstclosestxvector_2=0.6694456338882446, cells_neighbors_firstclosestyvector_2=-1.2334684133529663, cells_neighbors_numberofneighbors_2=-0.08251848816871643, cells_neighbors_secondclosestobjectnumber_2=0.010590200312435627, cells_nei

In [7]:
for i in range(2, 3):
    km = GaussianMixture(k=i, seed=23, probabilityCol="responsibilities")
    model = km.fit(df)
    print(i, model.summary.logLikelihood)

2 -905.6421484269327


In [72]:
a = model.gaussiansDF.select("mean").toPandas().values

In [75]:
a[2][0].values

IndexError: index 2 is out of bounds for axis 0 with size 2

In [32]:
a.

DataFrame[mean: vector]

In [28]:
numpy.array(a[1][0])

0.019724706977394

In [13]:
model.transform(df).take(1)

[Row(study='infectx_published', pathogen='listeria', library='a', design='u', replicate='1', plate='kb2-02-1x', well='a01', gene='chka', sirna='s3008', well_type='sirna', image_idx='4', object_idx='144', cells_areashape_area=-0.8044235706329346, cells_areashape_eccentricity=0.012107263319194317, cells_areashape_extent=1.115902304649353, cells_areashape_formfactor=1.474928617477417, cells_areashape_majoraxislength=-0.9368530511856079, cells_areashape_minoraxislength=-0.7485320568084717, cells_areashape_perimeter=-1.0209269523620605, cells_location_center_x=0.44758084416389465, cells_location_center_y=1.2808903455734253, cells_neighbors_anglebetweenneighbors_2=1.3915760517120361, cells_neighbors_firstclosestobjectnumber_2=0.1488679200410843, cells_neighbors_firstclosestxvector_2=0.6694456338882446, cells_neighbors_firstclosestyvector_2=-1.2334684133529663, cells_neighbors_numberofneighbors_2=-0.08251848816871643, cells_neighbors_secondclosestobjectnumber_2=0.010590200312435627, cells_nei

In [7]:
import scipy
from scipy.stats import multivariate_normal
svd = multivariate_normal.rvs(mean=scipy.ones(5), size=100)

In [172]:
df.take(1)

[Row(label=0, features=DenseVector([0.2978, 1.4635, 0.4609, -0.5065, 1.8492]))]

In [9]:
km = KMeans(seed=5, k=10)
model = km.fit(df)

In [151]:
model.summary.

[Row(label=0, features=DenseVector([1.2011, 0.7218, 0.0537, 2.2096]))]

In [41]:
km = GaussianMixture().setK(5).setSeed(23)
model = km.fit(df)

In [56]:
model.gaussiansDF

DataFrame[mean: vector, cov: matrix]

In [58]:
model.weights

[0.14620840581414205,
 0.15119357104416584,
 0.08541240437382741,
 0.255908435981313,
 0.3612771827865516]

In [125]:
import scipy
import sklearn.mixture
from scipy.stats import multivariate_normal
from sklearn.mixture import GaussianMixture

In [126]:
scipy.random.seed(23)
X = multivariate_normal.rvs(mean=scipy.ones(10), size=100)

In [127]:
dff = map(lambda x: (int(x[0]), Vectors.dense(x[0:])), X)
df = spark.createDataFrame(dff, schema=["label", "features"])

In [128]:
for i in [100, 90, 80, 70, 60, 50]:
    km = pyspark.ml.clustering.GaussianMixture(k=10, seed=23).fit(df.limit(i))
    sk_gmm = sklearn.mixture.GaussianMixture(10, random_state=23).fit(X[:i, :])
    print(df.limit(i).count(), X[:i, :].shape[0], km.summary.logLikelihood, sk_gmm.score(X[:i, :]))

100 100 368.37475644171036 -1.54949312502
90 90 1026.084529101155 1.16196607062
80 80 2245.427539835042 4.25769131857
70 70 1940.0122633489268 10.0949992881
60 60 2255.002313247103 14.0497823725
50 50 -140.82605873444814 21.2423016046


In [58]:
s = pyspark.ml.clustering.GaussianMixture(k=10, seed=23).fit(df.limit(100))

In [120]:
scipy.linalg.inv(s.gaussiansDF.select("cov").take(10)[10][0].toArray())

IndexError: list index out of range

In [22]:
sk_gmm.score_samples

<bound method BaseMixture.score_samples of GaussianMixture(covariance_type='full', init_params='kmeans', max_iter=100,
        means_init=None, n_components=10, n_init=1, precisions_init=None,
        random_state=None, reg_covar=1e-06, tol=0.001, verbose=0,
        verbose_interval=10, warm_start=False, weights_init=None)>